# ResNet





In [ ]:
pip install pandas scikit-learn openml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.6 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=3421a837639c37f486cafd46d2eda1cfe4ec8d9c7d3e6b970dc53d2652a6cc48
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff


In [ ]:
pip install torch torchvision

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load tabular data from OpenML with preprocessing
def load_openml_data(task_id):
    task = openml.tasks.get_task(task_id)
    dataset = openml.datasets.get_dataset(task.dataset_id)
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

    # Handle missing values
    imputer = SimpleImputer(strategy="most_frequent")
    X = imputer.fit_transform(X)

    # Convert categorical columns in X to numerical codes
    for col in range(X.shape[1]):
        if isinstance(X[0, col], str):
            X[:, col] = pd.factorize(X[:, col])[0]

    # Convert target y to numeric if needed and ensure it's a numpy array
    if isinstance(y[0], str):
        y = pd.factorize(y)[0]
    y = np.array(y)  # Ensure y is a numpy array

    return X.astype(np.float32), y.astype(np.int64)

# Define a custom dataset for tabular data
class TabularDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Define a simple ResNet-like model for tabular data
class ResNetTabular(nn.Module):
    def __init__(self, input_size, num_classes):
        super(ResNetTabular, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

# Get data loaders
def get_data_loaders(X, y, batch_size):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    train_dataset = TabularDataset(X_train, np.array(y_train))
    val_dataset = TabularDataset(X_val, np.array(y_val))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

# Task ID1: 14965

In [ ]:
# Parameters
task_id = 14965  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.4260
Epoch [2/10], Loss: 0.5946
Epoch [3/10], Loss: 0.1920
Epoch [4/10], Loss: 0.2464
Epoch [5/10], Loss: 0.0870
Epoch [6/10], Loss: 0.1267
Epoch [7/10], Loss: 0.0284
Epoch [8/10], Loss: 0.2418
Epoch [9/10], Loss: 0.1787
Epoch [10/10], Loss: 0.5847
Accuracy: 0.8986


# Task ID2: 9977

In [ ]:
# Parameters
task_id = 9977  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.0974
Epoch [2/10], Loss: 0.2079
Epoch [3/10], Loss: 0.2211
Epoch [4/10], Loss: 0.1062
Epoch [5/10], Loss: 0.2668
Epoch [6/10], Loss: 0.1894
Epoch [7/10], Loss: 0.0312
Epoch [8/10], Loss: 0.0508
Epoch [9/10], Loss: 0.2419
Epoch [10/10], Loss: 0.0775
Accuracy: 0.9569


# Task ID3: 34539

In [ ]:
# Parameters
task_id = 34539  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.0910
Epoch [2/10], Loss: 0.4356
Epoch [3/10], Loss: 0.0497
Epoch [4/10], Loss: 0.4178
Epoch [5/10], Loss: 0.0615
Epoch [6/10], Loss: 0.0547
Epoch [7/10], Loss: 0.4389
Epoch [8/10], Loss: 0.0719
Epoch [9/10], Loss: 0.0612
Epoch [10/10], Loss: 0.0526
Accuracy: 0.9437


# Task ID4: 146606

In [ ]:
# Parameters
task_id = 146606  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.7213
Epoch [2/10], Loss: 0.4284
Epoch [3/10], Loss: 0.6582
Epoch [4/10], Loss: 0.5026
Epoch [5/10], Loss: 0.6339
Epoch [6/10], Loss: 0.4914
Epoch [7/10], Loss: 0.3908
Epoch [8/10], Loss: 0.5078
Epoch [9/10], Loss: 0.4592
Epoch [10/10], Loss: 0.5521
Accuracy: 0.7110


# Task ID5: 7592



In [ ]:
# Parameters
task_id = 7592  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.0014
Epoch [2/10], Loss: 0.6058
Epoch [3/10], Loss: 0.8110
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0696
Epoch [6/10], Loss: 0.3649
Epoch [7/10], Loss: 0.7185
Epoch [8/10], Loss: 0.0277
Epoch [9/10], Loss: 0.0139
Epoch [10/10], Loss: 0.0697
Accuracy: 0.8547


# Task ID6: 146195



In [ ]:
# Parameters
task_id = 146195  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.8686
Epoch [2/10], Loss: 0.9432
Epoch [3/10], Loss: 0.6589
Epoch [4/10], Loss: 0.7128
Epoch [5/10], Loss: 0.8151
Epoch [6/10], Loss: 0.9076
Epoch [7/10], Loss: 0.5395
Epoch [8/10], Loss: 0.7531
Epoch [9/10], Loss: 0.4656
Epoch [10/10], Loss: 0.6060
Accuracy: 0.7419


# Task ID7: 167119



In [ ]:
# Parameters
task_id = 167119  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.7850
Epoch [2/10], Loss: 0.5707
Epoch [3/10], Loss: 0.5016
Epoch [4/10], Loss: 0.4219
Epoch [5/10], Loss: 0.5030
Epoch [6/10], Loss: 0.2640
Epoch [7/10], Loss: 0.3358
Epoch [8/10], Loss: 0.5580
Epoch [9/10], Loss: 0.3370
Epoch [10/10], Loss: 0.6294
Accuracy: 0.8168


# Task ID8: 167120



In [ ]:
# Parameters
task_id = 167120  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6991
Epoch [2/10], Loss: 0.6941
Epoch [3/10], Loss: 0.6848
Epoch [4/10], Loss: 0.6814
Epoch [5/10], Loss: 0.6823
Epoch [6/10], Loss: 0.7074
Epoch [7/10], Loss: 0.6843
Epoch [8/10], Loss: 0.6936
Epoch [9/10], Loss: 0.6967
Epoch [10/10], Loss: 0.6904
Accuracy: 0.5147


# Task ID11: 	168331



In [ ]:
# Parameters
task_id = 168331 # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.2453
Epoch [2/10], Loss: 1.2675
Epoch [3/10], Loss: 1.3478
Epoch [4/10], Loss: 1.6738
Epoch [5/10], Loss: 1.0183
Epoch [6/10], Loss: 0.6520
Epoch [7/10], Loss: 1.0676
Epoch [8/10], Loss: 1.2553
Epoch [9/10], Loss: 1.3280
Epoch [10/10], Loss: 1.3362
Accuracy: 0.6153


# Task ID12: 	168330



In [ ]:
# Parameters
task_id = 168330  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.9018
Epoch [2/10], Loss: 0.9688
Epoch [3/10], Loss: 1.2281
Epoch [4/10], Loss: 0.5237
Epoch [5/10], Loss: 0.6596
Epoch [6/10], Loss: 0.8224
Epoch [7/10], Loss: 0.5093
Epoch [8/10], Loss: 0.7616
Epoch [9/10], Loss: 0.5017
Epoch [10/10], Loss: 0.7088
Accuracy: 0.6976


# Task ID13: 	168335



In [ ]:
# Parameters
task_id = 168335  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.2640
Epoch [2/10], Loss: 0.3778
Epoch [3/10], Loss: 0.2037
Epoch [4/10], Loss: 0.4725
Epoch [5/10], Loss: 0.1526
Epoch [6/10], Loss: 0.2035
Epoch [7/10], Loss: 0.2507
Epoch [8/10], Loss: 0.0468
Epoch [9/10], Loss: 0.0393
Epoch [10/10], Loss: 0.0797
Accuracy: 0.9237



# Task ID16: 	146212



In [ ]:
# Parameters
task_id = 146212  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.0848
Epoch [2/10], Loss: 0.0027
Epoch [3/10], Loss: 0.0044
Epoch [4/10], Loss: 0.0607
Epoch [5/10], Loss: 0.0098
Epoch [6/10], Loss: 0.0027
Epoch [7/10], Loss: 0.0029
Epoch [8/10], Loss: 0.0078
Epoch [9/10], Loss: 0.0013
Epoch [10/10], Loss: 0.0029
Accuracy: 0.9967


# Task ID19: 	168868



In [ ]:
# Parameters
task_id = 168868  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.0231
Epoch [2/10], Loss: 0.0178
Epoch [3/10], Loss: 0.0001
Epoch [4/10], Loss: 0.0014
Epoch [5/10], Loss: 0.0202
Epoch [6/10], Loss: 0.0688
Epoch [7/10], Loss: 0.0147
Epoch [8/10], Loss: 0.0085
Epoch [9/10], Loss: 0.0034
Epoch [10/10], Loss: 0.0060
Accuracy: 0.9904


# Task ID20: 31

In [ ]:
# Parameters
task_id = 31  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6869
Epoch [2/10], Loss: 0.5056
Epoch [3/10], Loss: 0.5218
Epoch [4/10], Loss: 0.4387
Epoch [5/10], Loss: 0.5354
Epoch [6/10], Loss: 0.5089
Epoch [7/10], Loss: 0.6623
Epoch [8/10], Loss: 0.5546
Epoch [9/10], Loss: 0.5116
Epoch [10/10], Loss: 0.5411
Accuracy: 0.7700


# Task ID21: 10101

In [ ]:
# Parameters
task_id = 10101  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.5918
Epoch [2/10], Loss: 0.4164
Epoch [3/10], Loss: 0.4128
Epoch [4/10], Loss: 0.5102
Epoch [5/10], Loss: 0.5324
Epoch [6/10], Loss: 0.5481
Epoch [7/10], Loss: 0.4402
Epoch [8/10], Loss: 0.5139
Epoch [9/10], Loss: 0.4102
Epoch [10/10], Loss: 0.6876
Accuracy: 0.7667


# Task ID22: 	3913

In [ ]:
# Parameters
task_id = 3913  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.5112
Epoch [2/10], Loss: 0.2776
Epoch [3/10], Loss: 1.0583
Epoch [4/10], Loss: 0.2997
Epoch [5/10], Loss: 0.0267
Epoch [6/10], Loss: 0.0063
Epoch [7/10], Loss: 0.0171
Epoch [8/10], Loss: 0.0147
Epoch [9/10], Loss: 0.0334
Epoch [10/10], Loss: 0.0413
Accuracy: 0.8667


# Task ID23: 3

In [ ]:
# Parameters
task_id = 3 # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.3384
Epoch [2/10], Loss: 0.1961
Epoch [3/10], Loss: 0.1481
Epoch [4/10], Loss: 0.1710
Epoch [5/10], Loss: 0.1064
Epoch [6/10], Loss: 0.1497
Epoch [7/10], Loss: 0.0458
Epoch [8/10], Loss: 0.1258
Epoch [9/10], Loss: 0.0724
Epoch [10/10], Loss: 0.0192
Accuracy: 0.9672


# Task ID24: 3917

In [ ]:
# Parameters
task_id = 3917  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.4596
Epoch [2/10], Loss: 0.5719
Epoch [3/10], Loss: 0.4327
Epoch [4/10], Loss: 0.3523
Epoch [5/10], Loss: 0.2566
Epoch [6/10], Loss: 0.4604
Epoch [7/10], Loss: 0.4556
Epoch [8/10], Loss: 0.1894
Epoch [9/10], Loss: 0.3987
Epoch [10/10], Loss: 0.3939
Accuracy: 0.8555


# Task ID25: 9957

In [ ]:
# Parameters
task_id = 9957 # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6627
Epoch [2/10], Loss: 0.8670
Epoch [3/10], Loss: 0.4477
Epoch [4/10], Loss: 0.7525
Epoch [5/10], Loss: 0.3378
Epoch [6/10], Loss: 0.2250
Epoch [7/10], Loss: 0.1704
Epoch [8/10], Loss: 0.3046
Epoch [9/10], Loss: 0.1955
Epoch [10/10], Loss: 0.1029
Accuracy: 0.8768


# Task ID26: 9946

In [ ]:
# Parameters
task_id = 9946  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.5340
Epoch [2/10], Loss: 0.2510
Epoch [3/10], Loss: 0.1707
Epoch [4/10], Loss: 0.0688
Epoch [5/10], Loss: 0.0360
Epoch [6/10], Loss: 0.6044
Epoch [7/10], Loss: 0.1031
Epoch [8/10], Loss: 0.1066
Epoch [9/10], Loss: 0.0260
Epoch [10/10], Loss: 0.0106
Accuracy: 0.9737


# Task ID27: 3918

In [ ]:
# Parameters
task_id = 3918  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.3709
Epoch [2/10], Loss: 0.1950
Epoch [3/10], Loss: 0.2589
Epoch [4/10], Loss: 0.1441
Epoch [5/10], Loss: 0.1903
Epoch [6/10], Loss: 0.2888
Epoch [7/10], Loss: 0.2672
Epoch [8/10], Loss: 0.1748
Epoch [9/10], Loss: 0.1271
Epoch [10/10], Loss: 0.0570
Accuracy: 0.9234


# Task ID28: 3903

In [ ]:
# Parameters
task_id = 3903  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.1874
Epoch [2/10], Loss: 0.0021
Epoch [3/10], Loss: 0.0902
Epoch [4/10], Loss: 0.0500
Epoch [5/10], Loss: 0.2451
Epoch [6/10], Loss: 0.0912
Epoch [7/10], Loss: 0.2821
Epoch [8/10], Loss: 0.0349
Epoch [9/10], Loss: 0.0121
Epoch [10/10], Loss: 0.2675
Accuracy: 0.8946


# Task ID29: 37

In [ ]:
# Parameters
task_id = 37  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.7980
Epoch [2/10], Loss: 0.5303
Epoch [3/10], Loss: 0.4557
Epoch [4/10], Loss: 0.3613
Epoch [5/10], Loss: 0.2631
Epoch [6/10], Loss: 0.2889
Epoch [7/10], Loss: 0.7615
Epoch [8/10], Loss: 0.6689
Epoch [9/10], Loss: 0.3163
Epoch [10/10], Loss: 1.0312
Accuracy: 0.7727


# Task ID30: 9971

In [ ]:
# Parameters
task_id = 9971  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.7102
Epoch [2/10], Loss: 0.6663
Epoch [3/10], Loss: 0.6042
Epoch [4/10], Loss: 0.5484
Epoch [5/10], Loss: 0.5739
Epoch [6/10], Loss: 0.5683
Epoch [7/10], Loss: 0.4374
Epoch [8/10], Loss: 0.5640
Epoch [9/10], Loss: 0.4436
Epoch [10/10], Loss: 0.5529
Accuracy: 0.7521


# Task ID31: 9952

In [ ]:
# Parameters
task_id = 9952  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.4420
Epoch [2/10], Loss: 0.2358
Epoch [3/10], Loss: 0.6512
Epoch [4/10], Loss: 0.2106
Epoch [5/10], Loss: 0.0253
Epoch [6/10], Loss: 0.0595
Epoch [7/10], Loss: 0.2756
Epoch [8/10], Loss: 0.7425
Epoch [9/10], Loss: 0.1605
Epoch [10/10], Loss: 0.1029
Accuracy: 0.8409


# Task ID32: 3902

In [ ]:
# Parameters
task_id = 3902 # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.5660
Epoch [2/10], Loss: 0.2736
Epoch [3/10], Loss: 0.5322
Epoch [4/10], Loss: 0.1324
Epoch [5/10], Loss: 0.2318
Epoch [6/10], Loss: 0.0899
Epoch [7/10], Loss: 0.0418
Epoch [8/10], Loss: 0.1281
Epoch [9/10], Loss: 0.2423
Epoch [10/10], Loss: 0.1533
Accuracy: 0.9007


# Task ID33: 49

In [ ]:
# Parameters
task_id = 49  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6397
Epoch [2/10], Loss: 0.6337
Epoch [3/10], Loss: 0.6754
Epoch [4/10], Loss: 0.5163
Epoch [5/10], Loss: 0.5286
Epoch [6/10], Loss: 0.5523
Epoch [7/10], Loss: 0.5079
Epoch [8/10], Loss: 0.6817
Epoch [9/10], Loss: 0.4796
Epoch [10/10], Loss: 0.5917
Accuracy: 0.7448


# Task ID34: 43

In [ ]:
# Parameters
task_id = 43  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.3681
Epoch [2/10], Loss: 0.4118
Epoch [3/10], Loss: 0.1910
Epoch [4/10], Loss: 0.1333
Epoch [5/10], Loss: 0.3396
Epoch [6/10], Loss: 0.3177
Epoch [7/10], Loss: 0.2570
Epoch [8/10], Loss: 0.0715
Epoch [9/10], Loss: 0.1488
Epoch [10/10], Loss: 0.2670
Accuracy: 0.9414


# Task ID35: 9978

In [ ]:
# Parameters
task_id = 9978  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.2508
Epoch [2/10], Loss: 0.0895
Epoch [3/10], Loss: 0.2939
Epoch [4/10], Loss: 0.2479
Epoch [5/10], Loss: 0.1636
Epoch [6/10], Loss: 0.0165
Epoch [7/10], Loss: 0.0959
Epoch [8/10], Loss: 0.0246
Epoch [9/10], Loss: 0.0195
Epoch [10/10], Loss: 0.0221
Accuracy: 0.9428


# Task ID36: 10093

In [ ]:
# Parameters
task_id = 10093 # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.3740
Epoch [2/10], Loss: 0.1824
Epoch [3/10], Loss: 0.0304
Epoch [4/10], Loss: 0.0635
Epoch [5/10], Loss: 0.0438
Epoch [6/10], Loss: 0.3713
Epoch [7/10], Loss: 0.0670
Epoch [8/10], Loss: 0.0077
Epoch [9/10], Loss: 0.0743
Epoch [10/10], Loss: 0.0029
Accuracy: 0.9927


# Task ID37: 219

In [ ]:
# Parameters
task_id = 219  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.4165
Epoch [2/10], Loss: 0.5305
Epoch [3/10], Loss: 0.4529
Epoch [4/10], Loss: 0.4580
Epoch [5/10], Loss: 0.6772
Epoch [6/10], Loss: 0.5069
Epoch [7/10], Loss: 0.5180
Epoch [8/10], Loss: 0.2981
Epoch [9/10], Loss: 0.5443
Epoch [10/10], Loss: 0.4480
Accuracy: 0.7951


# Task ID38: 9976

In [ ]:
# Parameters
task_id = 9976  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6845
Epoch [2/10], Loss: 0.5947
Epoch [3/10], Loss: 0.4864
Epoch [4/10], Loss: 0.6458
Epoch [5/10], Loss: 0.4855
Epoch [6/10], Loss: 0.2363
Epoch [7/10], Loss: 0.2353
Epoch [8/10], Loss: 0.0765
Epoch [9/10], Loss: 0.3338
Epoch [10/10], Loss: 0.2892
Accuracy: 0.5731


# Task ID39: 6

In [ ]:
# Parameters
task_id = 6  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.9848
Epoch [2/10], Loss: 1.1941
Epoch [3/10], Loss: 1.0551
Epoch [4/10], Loss: 1.0708
Epoch [5/10], Loss: 1.2120
Epoch [6/10], Loss: 0.8768
Epoch [7/10], Loss: 1.1595
Epoch [8/10], Loss: 0.5226
Epoch [9/10], Loss: 1.2247
Epoch [10/10], Loss: 0.6530
Accuracy: 0.8377


# Task ID40: 53

In [ ]:
# Parameters
task_id = 53  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.3027
Epoch [2/10], Loss: 1.2649
Epoch [3/10], Loss: 0.9712
Epoch [4/10], Loss: 0.6766
Epoch [5/10], Loss: 0.8990
Epoch [6/10], Loss: 0.5625
Epoch [7/10], Loss: 0.7454
Epoch [8/10], Loss: 0.9923
Epoch [9/10], Loss: 0.5062
Epoch [10/10], Loss: 0.5359
Accuracy: 0.7824


# Task ID41: 11

In [ ]:
# Parameters
task_id = 11  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.9080
Epoch [2/10], Loss: 0.8733
Epoch [3/10], Loss: 0.8591
Epoch [4/10], Loss: 0.4565
Epoch [5/10], Loss: 0.5395
Epoch [6/10], Loss: 0.4304
Epoch [7/10], Loss: 0.2466
Epoch [8/10], Loss: 0.2302
Epoch [9/10], Loss: 0.3437
Epoch [10/10], Loss: 0.4615
Accuracy: 0.9040


# Task ID42: 15

In [ ]:
# Parameters
task_id = 15  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.4783
Epoch [2/10], Loss: 0.1655
Epoch [3/10], Loss: 0.3796
Epoch [4/10], Loss: 0.0724
Epoch [5/10], Loss: 0.0301
Epoch [6/10], Loss: 0.2333
Epoch [7/10], Loss: 0.6262
Epoch [8/10], Loss: 0.0076
Epoch [9/10], Loss: 0.0314
Epoch [10/10], Loss: 0.0277
Accuracy: 0.9714


# Task ID43: 16

In [ ]:
# Parameters
task_id = 16  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.9781
Epoch [2/10], Loss: 0.8679
Epoch [3/10], Loss: 0.6595
Epoch [4/10], Loss: 0.3748
Epoch [5/10], Loss: 0.2482
Epoch [6/10], Loss: 0.5316
Epoch [7/10], Loss: 0.1599
Epoch [8/10], Loss: 0.2725
Epoch [9/10], Loss: 0.1761
Epoch [10/10], Loss: 0.1800
Accuracy: 0.9575


# Task ID44: 14

In [ ]:
# Parameters
task_id = 14  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.9336
Epoch [2/10], Loss: 1.2328
Epoch [3/10], Loss: 0.7231
Epoch [4/10], Loss: 0.7118
Epoch [5/10], Loss: 0.5210
Epoch [6/10], Loss: 0.5873
Epoch [7/10], Loss: 0.6012
Epoch [8/10], Loss: 0.6615
Epoch [9/10], Loss: 0.3988
Epoch [10/10], Loss: 0.2812
Accuracy: 0.8275


# Task ID45: 32

In [ ]:
# Parameters
task_id = 32  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6213
Epoch [2/10], Loss: 0.2162
Epoch [3/10], Loss: 0.5217
Epoch [4/10], Loss: 0.2088
Epoch [5/10], Loss: 0.1713
Epoch [6/10], Loss: 0.0627
Epoch [7/10], Loss: 0.1875
Epoch [8/10], Loss: 0.2259
Epoch [9/10], Loss: 0.1253
Epoch [10/10], Loss: 0.1890
Accuracy: 0.9809


# Task ID46: 3549

In [ ]:
# Parameters
task_id = 3549  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.0427
Epoch [2/10], Loss: 0.6480
Epoch [3/10], Loss: 0.2173
Epoch [4/10], Loss: 0.3367
Epoch [5/10], Loss: 0.1164
Epoch [6/10], Loss: 0.0216
Epoch [7/10], Loss: 0.0136
Epoch [8/10], Loss: 0.0206
Epoch [9/10], Loss: 0.0112
Epoch [10/10], Loss: 0.0094
Accuracy: 1.0000


# Task ID47: 12

In [ ]:
# Parameters
task_id = 12 # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.0848
Epoch [2/10], Loss: 0.4446
Epoch [3/10], Loss: 0.3421
Epoch [4/10], Loss: 0.1097
Epoch [5/10], Loss: 0.2730
Epoch [6/10], Loss: 0.1099
Epoch [7/10], Loss: 0.1292
Epoch [8/10], Loss: 0.0717
Epoch [9/10], Loss: 0.1066
Epoch [10/10], Loss: 0.0456
Accuracy: 0.9750


# Task ID48: 9981

In [ ]:
# Parameters
task_id = 9981  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 2.0943
Epoch [2/10], Loss: 1.6230
Epoch [3/10], Loss: 0.9115
Epoch [4/10], Loss: 0.4620
Epoch [5/10], Loss: 0.3348
Epoch [6/10], Loss: 0.3186
Epoch [7/10], Loss: 0.0979
Epoch [8/10], Loss: 0.1229
Epoch [9/10], Loss: 0.1042
Epoch [10/10], Loss: 0.1168
Accuracy: 0.9491


# Task ID49: 18

In [ ]:
# Parameters
task_id = 18  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.7950
Epoch [2/10], Loss: 1.5099
Epoch [3/10], Loss: 1.0113
Epoch [4/10], Loss: 0.8657
Epoch [5/10], Loss: 0.7223
Epoch [6/10], Loss: 0.6450
Epoch [7/10], Loss: 0.7077
Epoch [8/10], Loss: 0.9311
Epoch [9/10], Loss: 0.9843
Epoch [10/10], Loss: 0.7082
Accuracy: 0.7300


# Task ID50: 28

In [ ]:
# Parameters
task_id = 28  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.2769
Epoch [2/10], Loss: 0.3782
Epoch [3/10], Loss: 0.4242
Epoch [4/10], Loss: 0.0945
Epoch [5/10], Loss: 0.2345
Epoch [6/10], Loss: 0.0353
Epoch [7/10], Loss: 0.0238
Epoch [8/10], Loss: 0.1468
Epoch [9/10], Loss: 0.0908
Epoch [10/10], Loss: 0.0219
Accuracy: 0.9751


# Task ID51: 2074

In [ ]:
# Parameters
task_id = 2074  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.2922
Epoch [2/10], Loss: 0.3533
Epoch [3/10], Loss: 0.4110
Epoch [4/10], Loss: 0.3735
Epoch [5/10], Loss: 0.4006
Epoch [6/10], Loss: 0.3744
Epoch [7/10], Loss: 0.1820
Epoch [8/10], Loss: 0.2762
Epoch [9/10], Loss: 0.2456
Epoch [10/10], Loss: 0.1159
Accuracy: 0.8919


# Task ID52: 29

In [ ]:
# Parameters
task_id = 29  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.5683
Epoch [2/10], Loss: 0.6107
Epoch [3/10], Loss: 0.4976
Epoch [4/10], Loss: 0.3759
Epoch [5/10], Loss: 0.2898
Epoch [6/10], Loss: 0.6791
Epoch [7/10], Loss: 0.4550
Epoch [8/10], Loss: 0.2221
Epoch [9/10], Loss: 0.1403
Epoch [10/10], Loss: 0.2874
Accuracy: 0.8333


# Task ID53: 45

In [ ]:
# Parameters
task_id = 45  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.8273
Epoch [2/10], Loss: 0.3736
Epoch [3/10], Loss: 0.4439
Epoch [4/10], Loss: 0.5267
Epoch [5/10], Loss: 0.2413
Epoch [6/10], Loss: 0.1165
Epoch [7/10], Loss: 0.2268
Epoch [8/10], Loss: 0.1592
Epoch [9/10], Loss: 0.2348
Epoch [10/10], Loss: 0.0930
Accuracy: 0.9028


# Task ID54: 125922

In [ ]:
# Parameters
task_id = 125922  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.7484
Epoch [2/10], Loss: 1.1360
Epoch [3/10], Loss: 0.4406
Epoch [4/10], Loss: 0.2962
Epoch [5/10], Loss: 0.2426
Epoch [6/10], Loss: 0.0788
Epoch [7/10], Loss: 0.0731
Epoch [8/10], Loss: 0.2455
Epoch [9/10], Loss: 0.0644
Epoch [10/10], Loss: 0.2277
Accuracy: 0.9809


# Task ID55: 9960

In [ ]:
# Parameters
task_id = 9960 # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.1461
Epoch [2/10], Loss: 0.6053
Epoch [3/10], Loss: 1.0406
Epoch [4/10], Loss: 0.6747
Epoch [5/10], Loss: 0.5201
Epoch [6/10], Loss: 0.9662
Epoch [7/10], Loss: 0.6516
Epoch [8/10], Loss: 0.2981
Epoch [9/10], Loss: 0.7435
Epoch [10/10], Loss: 0.2616
Accuracy: 0.8233


# Task ID56: 9964

In [ ]:
# Parameters
task_id = 9964  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.7351
Epoch [2/10], Loss: 0.8990
Epoch [3/10], Loss: 0.5939
Epoch [4/10], Loss: 0.3084
Epoch [5/10], Loss: 0.3122
Epoch [6/10], Loss: 0.3787
Epoch [7/10], Loss: 0.1843
Epoch [8/10], Loss: 0.3635
Epoch [9/10], Loss: 0.2732
Epoch [10/10], Loss: 0.1935
Accuracy: 0.9122


# Task ID57: 22

In [ ]:
# Parameters
task_id = 22  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.9153
Epoch [2/10], Loss: 1.1205
Epoch [3/10], Loss: 0.8081
Epoch [4/10], Loss: 0.8976
Epoch [5/10], Loss: 0.7231
Epoch [6/10], Loss: 0.9096
Epoch [7/10], Loss: 0.4405
Epoch [8/10], Loss: 0.6210
Epoch [9/10], Loss: 0.4733
Epoch [10/10], Loss: 0.6931
Accuracy: 0.8100


# Task ID58: 2079

In [ ]:
# Parameters
task_id = 2079  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.5049
Epoch [2/10], Loss: 1.3901
Epoch [3/10], Loss: 1.4932
Epoch [4/10], Loss: 1.3889
Epoch [5/10], Loss: 1.1976
Epoch [6/10], Loss: 0.9845
Epoch [7/10], Loss: 1.0315
Epoch [8/10], Loss: 0.9093
Epoch [9/10], Loss: 1.1634
Epoch [10/10], Loss: 1.2318
Accuracy: 0.5743


# Task ID59: 14969

In [ ]:
# Parameters
task_id = 14969  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.2482
Epoch [2/10], Loss: 1.2744
Epoch [3/10], Loss: 1.2302
Epoch [4/10], Loss: 1.1223
Epoch [5/10], Loss: 1.0441
Epoch [6/10], Loss: 1.0920
Epoch [7/10], Loss: 1.2609
Epoch [8/10], Loss: 1.2281
Epoch [9/10], Loss: 1.5800
Epoch [10/10], Loss: 1.1081
Accuracy: 0.5337


# Task ID60: 3560

In [ ]:
# Parameters
task_id = 3560  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.7975
Epoch [2/10], Loss: 1.7885
Epoch [3/10], Loss: 1.7737
Epoch [4/10], Loss: 1.7932
Epoch [5/10], Loss: 1.7920
Epoch [6/10], Loss: 1.7832
Epoch [7/10], Loss: 1.7472
Epoch [8/10], Loss: 1.7870
Epoch [9/10], Loss: 1.7094
Epoch [10/10], Loss: 1.7713
Accuracy: 0.2625


# Task ID61: 14952

In [ ]:
# Parameters
task_id = 14952  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.3069
Epoch [2/10], Loss: 0.1478
Epoch [3/10], Loss: 0.2428
Epoch [4/10], Loss: 0.1103
Epoch [5/10], Loss: 0.0717
Epoch [6/10], Loss: 0.1554
Epoch [7/10], Loss: 0.3517
Epoch [8/10], Loss: 0.2146
Epoch [9/10], Loss: 0.0141
Epoch [10/10], Loss: 0.0763
Accuracy: 0.9457


# Task ID62: 125920

In [ ]:
# Parameters
task_id = 125920  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6561
Epoch [2/10], Loss: 0.6528
Epoch [3/10], Loss: 0.6298
Epoch [4/10], Loss: 0.6924
Epoch [5/10], Loss: 0.6469
Epoch [6/10], Loss: 0.6853
Epoch [7/10], Loss: 0.7803
Epoch [8/10], Loss: 0.7304
Epoch [9/10], Loss: 0.7056
Epoch [10/10], Loss: 0.6113
Accuracy: 0.5800


# Task ID63: 23

In [ ]:
# Parameters
task_id = 23  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.9581
Epoch [2/10], Loss: 0.9892
Epoch [3/10], Loss: 0.9708
Epoch [4/10], Loss: 0.9396
Epoch [5/10], Loss: 1.0600
Epoch [6/10], Loss: 1.0222
Epoch [7/10], Loss: 0.8828
Epoch [8/10], Loss: 1.0116
Epoch [9/10], Loss: 0.8553
Epoch [10/10], Loss: 0.8614
Accuracy: 0.5661


# Task ID64: 3904

In [ ]:
# Parameters
task_id = 3904  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.2033
Epoch [2/10], Loss: 0.2305
Epoch [3/10], Loss: 0.1396
Epoch [4/10], Loss: 0.4330
Epoch [5/10], Loss: 0.5676
Epoch [6/10], Loss: 0.1745
Epoch [7/10], Loss: 0.2576
Epoch [8/10], Loss: 0.3292
Epoch [9/10], Loss: 0.9026
Epoch [10/10], Loss: 0.1090
Accuracy: 0.8112


# Task ID65: 3022

In [ ]:
# Parameters
task_id = 3022  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 2.3624
Epoch [2/10], Loss: 2.3273
Epoch [3/10], Loss: 2.0606
Epoch [4/10], Loss: 1.8122
Epoch [5/10], Loss: 1.6415
Epoch [6/10], Loss: 1.5427
Epoch [7/10], Loss: 1.5094
Epoch [8/10], Loss: 1.1993
Epoch [9/10], Loss: 1.2095
Epoch [10/10], Loss: 1.0983
Accuracy: 0.6263


# Task ID66: 9985

In [ ]:
# Parameters
task_id = 9985  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.3718
Epoch [2/10], Loss: 1.3923
Epoch [3/10], Loss: 1.4622
Epoch [4/10], Loss: 1.2558
Epoch [5/10], Loss: 1.3364
Epoch [6/10], Loss: 1.1151
Epoch [7/10], Loss: 1.4405
Epoch [8/10], Loss: 1.4199
Epoch [9/10], Loss: 1.1781
Epoch [10/10], Loss: 1.5860
Accuracy: 0.4943


# Task ID67: 9910

In [ ]:
# Parameters
task_id = 9910  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.4531
Epoch [2/10], Loss: 0.3834
Epoch [3/10], Loss: 0.5013
Epoch [4/10], Loss: 0.3747
Epoch [5/10], Loss: 0.5048
Epoch [6/10], Loss: 0.4651
Epoch [7/10], Loss: 0.6111
Epoch [8/10], Loss: 0.3295
Epoch [9/10], Loss: 0.2774
Epoch [10/10], Loss: 0.2270
Accuracy: 0.7843


# Task ID68: 14970

In [ ]:
# Parameters
task_id = 14970  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.1258
Epoch [2/10], Loss: 0.0953
Epoch [3/10], Loss: 0.1113
Epoch [4/10], Loss: 0.0469
Epoch [5/10], Loss: 0.0619
Epoch [6/10], Loss: 0.0536
Epoch [7/10], Loss: 0.0102
Epoch [8/10], Loss: 0.0102
Epoch [9/10], Loss: 0.0825
Epoch [10/10], Loss: 0.2505
Accuracy: 0.9699


# Task ID69: 3021

In [ ]:
# Parameters
task_id = 3021  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Epoch [1/10], Loss: nan
Epoch [2/10], Loss: nan
Epoch [3/10], Loss: nan
Epoch [4/10], Loss: nan
Epoch [5/10], Loss: nan
Epoch [6/10], Loss: nan
Epoch [7/10], Loss: nan
Epoch [8/10], Loss: nan
Epoch [9/10], Loss: nan
Epoch [10/10], Loss: nan
Accuracy: 0.9497


# Task ID70: 3481

In [ ]:
# Parameters
task_id = 3481  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6701
Epoch [2/10], Loss: 0.5188
Epoch [3/10], Loss: 0.4814
Epoch [4/10], Loss: 0.3009
Epoch [5/10], Loss: 0.4821
Epoch [6/10], Loss: 0.1372
Epoch [7/10], Loss: 0.1265
Epoch [8/10], Loss: 0.1033
Epoch [9/10], Loss: 0.1955
Epoch [10/10], Loss: 0.4077
Accuracy: 0.9519


# Task ID71: 3573

In [ ]:
# Parameters
task_id = 9946  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


# Task ID72: 146824

In [ ]:
# Parameters
task_id = 146824  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.0078
Epoch [2/10], Loss: 0.2178
Epoch [3/10], Loss: 0.2980
Epoch [4/10], Loss: 0.1871
Epoch [5/10], Loss: 0.1209
Epoch [6/10], Loss: 0.2171
Epoch [7/10], Loss: 0.1167
Epoch [8/10], Loss: 0.1056
Epoch [9/10], Loss: 0.0560
Epoch [10/10], Loss: 0.1168
Accuracy: 0.9625


# Task ID73: 146820

In [ ]:
# Parameters
task_id = 146820  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.1358
Epoch [2/10], Loss: 0.1010
Epoch [3/10], Loss: 0.0912
Epoch [4/10], Loss: 0.3462
Epoch [5/10], Loss: 0.1265
Epoch [6/10], Loss: 0.3241
Epoch [7/10], Loss: 0.0289
Epoch [8/10], Loss: 0.0818
Epoch [9/10], Loss: 0.0911
Epoch [10/10], Loss: 0.0320
Accuracy: 0.9783


# Task ID74: 146822

In [ ]:
# Parameters
task_id = 146822  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.3144
Epoch [2/10], Loss: 0.6705
Epoch [3/10], Loss: 0.4327
Epoch [4/10], Loss: 0.6459
Epoch [5/10], Loss: 0.4002
Epoch [6/10], Loss: 0.4064
Epoch [7/10], Loss: 0.5871
Epoch [8/10], Loss: 0.5033
Epoch [9/10], Loss: 0.2537
Epoch [10/10], Loss: 0.4168
Accuracy: 0.8636


# Task ID75: 146195

In [ ]:
# Parameters
task_id = 146195  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.7639
Epoch [2/10], Loss: 0.7727
Epoch [3/10], Loss: 0.8116
Epoch [4/10], Loss: 1.0752
Epoch [5/10], Loss: 0.9051
Epoch [6/10], Loss: 0.5998
Epoch [7/10], Loss: 0.7801
Epoch [8/10], Loss: 0.7414
Epoch [9/10], Loss: 0.8102
Epoch [10/10], Loss: 0.8910
Accuracy: 0.7438


# Task ID76: 146800

In [ ]:
# Parameters
task_id = 146800  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.8622
Epoch [2/10], Loss: 1.4900
Epoch [3/10], Loss: 1.1354
Epoch [4/10], Loss: 0.9604
Epoch [5/10], Loss: 0.6071
Epoch [6/10], Loss: 0.6917
Epoch [7/10], Loss: 0.4655
Epoch [8/10], Loss: 0.5691
Epoch [9/10], Loss: 0.3564
Epoch [10/10], Loss: 0.3355
Accuracy: 0.9537


# Task ID77: 146817

In [ ]:
# Parameters
task_id = 146817  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 1.4363
Epoch [2/10], Loss: 1.5182
Epoch [3/10], Loss: 0.9396
Epoch [4/10], Loss: 0.9890
Epoch [5/10], Loss: 1.1000
Epoch [6/10], Loss: 0.7605
Epoch [7/10], Loss: 0.4802
Epoch [8/10], Loss: 0.8595
Epoch [9/10], Loss: 1.2034
Epoch [10/10], Loss: 0.4989
Accuracy: 0.7224


# Task ID78: 146819

In [ ]:
# Parameters
task_id = 146819  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.5807
Epoch [2/10], Loss: 0.1377
Epoch [3/10], Loss: 0.5074
Epoch [4/10], Loss: 0.2223
Epoch [5/10], Loss: 0.0579
Epoch [6/10], Loss: 0.0759
Epoch [7/10], Loss: 0.1981
Epoch [8/10], Loss: 0.0467
Epoch [9/10], Loss: 0.1606
Epoch [10/10], Loss: 0.0927
Accuracy: 0.9074


# Task ID79: 146821

In [ ]:
# Parameters
task_id = 146821  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.9359
Epoch [2/10], Loss: 0.5941
Epoch [3/10], Loss: 0.6852
Epoch [4/10], Loss: 0.3264
Epoch [5/10], Loss: 0.1279
Epoch [6/10], Loss: 0.3459
Epoch [7/10], Loss: 0.5113
Epoch [8/10], Loss: 0.6479
Epoch [9/10], Loss: 0.0682
Epoch [10/10], Loss: 0.4343
Accuracy: 0.8931


# Task ID80: 14954

In [ ]:
# Parameters
task_id = 14954  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.6958
Epoch [2/10], Loss: 0.6255
Epoch [3/10], Loss: 0.6655
Epoch [4/10], Loss: 0.5182
Epoch [5/10], Loss: 0.4262
Epoch [6/10], Loss: 0.4989
Epoch [7/10], Loss: 0.5278
Epoch [8/10], Loss: 0.5725
Epoch [9/10], Loss: 0.3996
Epoch [10/10], Loss: 0.5399
Accuracy: 0.7315


# Task ID81: 167141

In [ ]:
# Parameters
task_id = 167141  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.3436
Epoch [2/10], Loss: 0.3594
Epoch [3/10], Loss: 0.1749
Epoch [4/10], Loss: 0.4268
Epoch [5/10], Loss: 0.5808
Epoch [6/10], Loss: 0.3436
Epoch [7/10], Loss: 0.3714
Epoch [8/10], Loss: 0.1146
Epoch [9/10], Loss: 0.2928
Epoch [10/10], Loss: 0.2401
Accuracy: 0.9260


# Task ID82: 167140

In [ ]:
# Parameters
task_id = 167140  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.5412
Epoch [2/10], Loss: 0.1624
Epoch [3/10], Loss: 0.1953
Epoch [4/10], Loss: 0.1324
Epoch [5/10], Loss: 0.0358
Epoch [6/10], Loss: 0.2913
Epoch [7/10], Loss: 0.2629
Epoch [8/10], Loss: 0.0061
Epoch [9/10], Loss: 0.0200
Epoch [10/10], Loss: 0.0457
Accuracy: 0.9498


# Task ID83: 167125

In [ ]:
# Parameters
task_id = 167125  # Example task ID from OpenML
batch_size = 32
epochs = 10
learning_rate = 0.001

# Load data
X, y = load_openml_data(task_id)

# Initialize data loaders
train_loader, val_loader = get_data_loaders(X, y, batch_size=batch_size)

# Initialize model, loss, and optimizer
num_classes = len(np.unique(y))
input_size = X.shape[1]  # Number of features
model = ResNetTabular(input_size=input_size, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for features, labels in val_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')


Epoch [1/10], Loss: 0.2874
Epoch [2/10], Loss: 0.0253
Epoch [3/10], Loss: 0.0332
Epoch [4/10], Loss: 0.0362
Epoch [5/10], Loss: 0.0667
Epoch [6/10], Loss: 0.0069
Epoch [7/10], Loss: 0.0061
Epoch [8/10], Loss: 0.0323
Epoch [9/10], Loss: 0.0654
Epoch [10/10], Loss: 0.0032
Accuracy: 0.9680


# Task ID84: 167124

# Task ID85: 167121

In [ ]:
pip freeze > requirements.txt